Import necessary libraries

In [38]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

In [39]:
# from Corey Schafer web scraping tutorial
# source = requests.get('http://coreyms.com').text

# soup = BeautifulSoup(source, 'lxml')

# csv_file = open('cms_scrape.csv', 'w')

# csv_writer = csv.writer(csv_file)
# csv_writer.writerow(['headline', 'summary', 'video_link'])

# for article in soup.find_all('article'):
#     headline = article.h2.a.text
#     print(headline)

#     summary = article.find('div', class_='entry-content').p.text
#     print(summary)

#     try:
#         vid_src = article.find('iframe', class_='youtube-player')['src']

#         vid_id = vid_src.split('/')[4]
#         vid_id = vid_id.split('?')[0]

#         yt_link = f'https://youtube.com/watch?v={vid_id}'
#     except Exception as e:
#         yt_link = None

#     print(yt_link)

#     print()

#     csv_writer.writerow([headline, summary, yt_link])

# csv_file.close()

Get that webpage and do some basic actions with it

In [40]:
# idea for web scraping: scrape audible best-selling books
URL = "https://www.audible.com/search?sort=popularity-rank&ref=a_search_l1_catBackAll&pf_rd_p=daf0f1c8-2865-4989-87fb-15115ba5a6d2&pf_rd_r=S4VEHTG7BAZHCPTD9YB7"
webpage = requests.get(URL)
soup = BeautifulSoup(webpage.text, "html.parser")

In [54]:
# many thanks to https://jovian.ai/pratulofficialthings/audible-business-and-careers-books-2022
book_list = soup.find_all('li', class_='bc-list-item productListItem')


def get_book_names(book_list):
    book_names = []
    for tag in book_list:
        a_tag_name = tag.h3.find_all('a', recursive=False)
        book_name = a_tag_name[0].text.strip()
        book_names.append(book_name)
    return book_names


def get_book_length(book_list):
    book_length = []
    for tag in book_list:
        try:
            len_tag = tag.find('li', class_='bc-list-item runtimeLabel')
            length_tag = len_tag.find('span')
            length = length_tag.text.strip()
            book_length.append(length)
        except AttributeError:
            book_length.append(None)
    # Remove "Length: " from every elem of output list
    book_length = [i[8:] for i in book_length]
    return book_length


def get_author(book_contents):
    written_by=[]
    for tag in book_contents:
        author_tag= tag.find('li', class_='bc-list-item authorLabel')
        try:
            auth_tag = author_tag.find('a')
            author = auth_tag.text.strip()
            written_by.append(author)
        except AttributeError:
            written_by.append(None)
    return written_by


def get_description(book_list):
    description = []
    for tag in book_list:
        about_tag = tag.find('li', class_='bc-list-item subtitle')
        try:
            description_tag = about_tag.find('span').text.strip()
            description.append(description_tag)
        except AttributeError:
            description.append(None)
    return description


def get_language(book_list):
    language_list = []
    for tag in book_list:
        lang_tag = tag.find('li', class_='bc-list-item languageLabel')
        try:
            language_tag = lang_tag.find('span').text.split()
            # Access 'English' in ['Language:', 'English'] of language_tag
            language_itself = language_tag[1]
            language_list.append(language_itself)
        except AttributeError:
            language_list.append(None)
    return language_list


def get_rating(book_list):
    rating = []
    for tag in book_list:
        star_tag = tag.find('li', class_='bc-list-item ratingsLabel')
        try:
            rating_tag = star_tag.find(
                'span', class_='bc-text bc-pub-offscreen').text.strip()
            rating.append(rating_tag)
        except AttributeError:
            rating.append(None)
    return rating


def get_nunber_of_ratings(book_list):
    nunber_of_ratings = []
    for tag in book_list:
        star_tag = tag.find('li', class_='bc-list-item ratingsLabel')
        try:
            rating_tag = star_tag.find(
                'span', class_='bc-text bc-size-small bc-color-secondary').text.strip()
            nunber_of_ratings.append(rating_tag)
        except AttributeError:
            nunber_of_ratings.append(None)
    return nunber_of_ratings


def get_regular_price(book_list):
    regular_price = []
    for tag in book_list:
        buy_tag = tag.find(
            'p', class_='bc-text buybox-regular-price bc-spacing-none bc-spacing-top-none')
        try:
            price_tag = buy_tag.find_all(
                'span', class_='bc-text bc-size-base bc-color-base')
            price = price_tag[1].text.strip()
            regular_price.append(price)
        except AttributeError:
            regular_price.append(None)
    return regular_price

df = pd.DataFrame(
{
    "Book_Name": get_book_names(book_list),
    "Description": get_description(book_list),
    "Author": get_author(book_list),
    "Rating": get_rating(book_list),
    "Num_of_Ratings": get_nunber_of_ratings(book_list),
    "Regular_Price": get_regular_price(book_list),
    "Audio_Length": get_book_length(book_list)
})

df

,Book_Name,Description,Author,Rating,Num_of_Ratings,Regular_Price,Audio_Length
0,"Friends, Lovers, and the Big Terrible Thing",A Memoir,Matthew Perry,4.5 out of 5 stars,"2,190 ratings",$25.51,8 hrs and 49 mins
1,It Starts with Us,A Novel,Colleen Hoover,4.5 out of 5 stars,"12,067 ratings",$23.62,8 hrs and 41 mins
2,Fairy Tale,None,Stephen King,5 out of 5 stars,"33,624 ratings",$26.90,24 hrs and 6 mins
3,I'm Glad My Mom Died,None,Jennette McCurdy,5 out of 5 stars,"68,555 ratings",$13.99,6 hrs and 26 mins
4,The Boys from Biloxi,A Legal Thriller,John Grisham,4.5 out of 5 stars,"2,263 ratings",$35.00,17 hrs and 22 mins
5,Atomic Habits,An Easy & Proven Way to Build Good Habits & Br...,James Clear,5 out of 5 stars,"112,358 ratings",$16.72,5 hrs and 35 mins
6,It Ends with Us,None,Colleen Hoover,4.5 out of 5 stars,"52,474 ratings",$13.49,11 hrs and 11 mins
7,Hebrews to Negroes,Wake Up Black America,Ronald Dalton Jr.,4.5 out of 5 stars,108 ratings,$29.95,36 hrs and 2 mins
8,Surrender,"40 Songs, One Story",Bono,5 out of 5 stars,87 ratings,$45.50,20 hrs and 25 mins
9,No Plan B,A Jack Reacher Novel,Lee Child,4.5 out of 5 stars,"1,507 ratings",$35.00,11 hrs and 3 mins
